**1. Deep Neural Nerwork - 다층 퍼셉트론 구현**

In [ ]:
# torch는 딥러닝을 구성하기 편하도록 페이스북에서 만든 딥러닝 라이브러리
# 텐서플로우랑 똑같은 딥러닝 라이브러리
import torch

# 신경망 구축을 쉽게 해주는 함수
import torch.nn as nn

In [ ]:
# GPU 확인
# GPU가 없으면 CPU로 연산
# GPU가 있으면 GPU로 연산하겠다는 준비
# 따라서 device는 CPU나 GPU 문자열로 바뀐다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# 4명의 데이터가 있는데, 1명당 2개의 컬럼을 가지고 있다.
# 그리고 그걸 2차원 리스트로 4명의 데이터를 선언 후
# 토치가 알아먹을 수 있는 텐서타입으로 바꾼다. 이때 텐서타입은 float
# to(device)는 모델이 GPU에서 연산될 것이기 떄문에 to(cuba)를 함으로써 데이터를 GPU에 얹혀 놓는다.
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [ ]:
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
    nn.Sigmoid(),
    nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
    nn.Sigmoid()
).to(device)

In [ ]:
criterion = torch.nn.MSELoss().to(device) # Mean Square Error
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# 옵티마이저인 Stochastic Gradient Descent를 사용한다.
# 업데이트 대상은 model 객체가 가지고 있는 W들을 대상으로 업데이트를 할 것이고, learning rate는 0.001로 설정할 것이다.

In [ ]:
for epoch in range(50): # 50번 학습할 것이다.
  optimizer.zero_grad() # 옵티마이저의 기울기를 매번 초기화 해서 계산해야하기 때문에 초기화하는 함수를 선언
  # forward 연산
  hypothesis = model(X) # 모델에 데이터를 넣고, propagation을 진행한 후, 최종 아웃풋을 hypothesis에 담는다.

  # 비용 함수
  cost = criterion(hypothesis, Y) # 예측값과 정답값 사이의 Mean Square Loss를 구한다.
  cost.backward() # backpropagtion이 아니다. 이름에 낚이지 말자. 이 부분은 기울기를 구하는 함수다.
  optimizer.step() # 여기서 비로소 backpropagation을 실행하여 모델의 모든 w들을 업데이트 한다.
  
  # 5의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 5 == 0:
    print("에폭 :", epoch, cost.item())

In [ ]:
X_2 = torch.FloatTensor([[0, 0], [0, 1]]).to(device)
hypothesis = model(X_2)
print(hypothesis)

**2. minibatch DNN**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [ ]:
# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# 5명의 데이터, 각 데이터는 3개의 컬럼(피처)를 갖는다.
x_train = torch.FloatTensor(
    [[73, 80, 75],
    [93, 88, 93],
    [89, 91, 90],
    [96, 98, 100],
    [73, 66, 70]]
)

# 정답값은 1사람당 1개
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [ ]:
# X 데이터셋과 Y 데이터 셋을 포장한다.
dataset = TensorDataset(x_train, y_train)
print(list(dataset))

In [ ]:
# 2개 데이터를 1세트로 만들어서 전체데이터셋을 쪼갠다. 이때 만든 n개의 세트들의 순서를 shuffle 한다.
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
model = nn.Sequential(
    nn.Linear(3, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
    nn.ReLU(),
    nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
    nn.ReLU(),
    nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
    nn.ReLU(),
    nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
    nn.ReLU()
).to(device)

In [ ]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 0.00001

In [ ]:
nb_epochs = 5

# 전체 데이터 셋으로 5번 학습할 것이다.
for epoch in range(nb_epochs + 1):
  # minibatch 데이터를 반환하면서 index도 반환한다.GPU 터질까봐 mini batch를 한다.
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    x_train= x_train.to(device)
    y_train= y_train.to(device)

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = criterion(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print("Epoch : {:4d}/{} Batch {}/{} Cost : {:.6f}".format(
      epoch, nb_epochs, batch_idx+1, len(dataloader),
      cost.item()
    ))

In [ ]:
X_2 = torch.FloatTensor([[100, 70, 60], [150, 30,75]]).to(device)
hypothesis = model(X_2)
print(hypothesis)

**3.CNN**

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 랜덤 시드 고정
torch.manual_seed(777)

In [ ]:
learning_rate = 0.001 # 옵티마이저의 learning rate
training_epochs = 15 # 학습횟수
batch_size = 100 # minibatch 사이즈, 전체 데이터셋에서 서브 셋을 몇개로 묶을 것인지

In [ ]:
mnist_train = dsets.MNIST(
    root="MNIST_data/", # 다운로드 경로 지정
    train=True, # True를 지정하면 훈련 데이터로 다운로드
    transform=transforms.ToTensor(), # 텐서로 변환, 토치에서 알아듣는 타입으로 지정
    download=True
)

mnist_test = dsets.MNIST(
    root="MNIST_data/", # 다운로드 경로 지정
    train=False, # False를 지정하면 테스트 데이터로 다운로드
    transform=transforms.ToTensor(), # 텐서로 변환, 토치에서 알아듣는 타입으로 지정
    download=True
)

In [ ]:
# drop_last 마지막 남은 데이터를 버릴 것인지, 버리지 않을 것인지
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [ ]:
# torch.nn.Module : PyTorch의 모든 Neural Network의 Base Class
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # 첫 번째 층
    # ImgIn shape = (?, 28, 28, 1)
    # Conv -> (?, 28, 28, 32)
    # Pool -> (?, 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 두 번째 층
    # ImgIn shape = (?, 14, 14, 32)
    # Conv -> (?, 14, 14, 64)
    # Pool -> (?, 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 전결합층 7x7x64 inputs -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

    # 전결합층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten
    out = self.fc(out)
    return out

In [ ]:
# CNN 모델 정의
model = CNN().to(device)

In [ ]:
# 비용 함수에 소프트맥스 함수 포함되어 있다.
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)
print("총 배치의 수 : {}".format(total_batch))

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0

  # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블
  for X, Y in data_loader:
    # image is already size of (28x28), no reshape
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("[Epoch : {:>4}] cost : {:>.9}".format(epoch + 1, avg_cost))

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)

  # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy :", accuracy.item())

In [ ]:
# 모델 저장
torch.save(model.state_dict(), "cnn_model.pt")

**3-1. 다시불러와서 추론 해보기**

In [ ]:
# 다시 불러와서 추론 해보기
model = CNN().to(device)
model.load_state_dict(torch.load("cnn_model.pt"))

# 평가 모드로 설정하여야 합니다. 이 과정을 거치지 않으면 일관성 없는 추론 결과가 출력
model.eval()

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  
  # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()

  print("Accuracy :", accuracy.item())

**3-2. 이미지 불러서 추론해보기**

In [ ]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

with torch.no_grad():
  # 이미지 파일 경로 설정
  img = Image.open("./8.png")
  
  transform = transforms.Compose([
      transforms.Grayscale(num_output_channels=1), # RGB(3D) -> Gray(2D)
      transforms.Resize((28, 28)), # 모델 인풋에 맞게
      transforms.ToTensor(), # 토치 텐서 타입으로 맞춰줘야한다.
  ])

  img_tensor = transform(img).to(device) # [1, 28, 28]
  img_tensor = img_tensor.unsqueeze(0) # [1, 1, 28, 28] # 모델이 원래 [배치사이즈, 채널, 가로, 세로]

  print(img_tensor.shape)

  prediction = model(img_tensor)

  # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
  print('result :', torch.argmax(prediction, 1))

  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  
  print("Accuracy :", accuracy.item())